# Normalize Reference Data (Consolidated)

**Purpose**: Creates and normalizes vendor/data stream reference data using token overlap matching.

This notebook is the **single source of truth** for reference data population. It runs AFTER all document processing (tsDTA + OA) to:
1. Create reference tables (`md_vendor`, `md_data_stream`) if they don't exist
2. Collect ALL vendor/data_stream names from `md_dta_excel_file_raw` and `md_oa_file_raw`
3. Apply token overlap clustering to group similar names (e.g., 'Labcorp CLS' ≈ 'Labcorp')
4. Use MERGE to INSERT new records or UPDATE aliases for existing ones
5. Support DTA creation with normalized values

**Input**:
- `bronze_md.md_dta_excel_file_raw` - tsDTA sheet metadata
- `bronze_md.md_oa_file_raw` - OA document content (if exists)

**Output**:
- `gold_md.md_vendor` - Vendor registry with normalized names and aliases
- `gold_md.md_data_stream` - Data stream registry with normalized names and aliases
- `gold_md.md_study` - Study registry (from trial_id)


In [ ]:
# Cell 1: Imports
import json
import uuid
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp, collect_list, collect_set

from clinical_data_standards_framework.utils import (
    tokenize_name,
    calculate_token_overlap,
    cluster_similar_names,
    find_canonical_name
)

print("✅ Framework loaded successfully")


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
gold_schema = globals_dict['gold_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Define Table Names
excel_raw_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
oa_raw_table = f"{catalog}.{bronze_schema}.md_oa_file_raw"
vendor_table = f"{catalog}.{gold_schema}.md_vendor"
data_stream_table = f"{catalog}.{gold_schema}.md_data_stream"
study_table = f"{catalog}.{gold_schema}.md_study"

print(f"Excel Raw table: {excel_raw_table}")
print(f"OA Raw table: {oa_raw_table}")
print(f"Vendor table: {vendor_table}")
print(f"Data Stream table: {data_stream_table}")
print(f"Study table: {study_table}")


In [ ]:
# Cell 3b: Create Reference Tables if Not Exist
# This notebook is the SINGLE SOURCE for reference data creation

print("\n" + "=" * 60)
print("Ensuring reference tables exist...")
print("=" * 60)

# Create md_vendor table if it doesn't exist
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {vendor_table} (
        vendor_id STRING NOT NULL COMMENT 'Unique vendor identifier',
        vendor_name STRING NOT NULL COMMENT 'Canonical vendor name (shortest/normalized)',
        aliases ARRAY<STRING> COMMENT 'All variant names mapped to this vendor',
        vendor_description STRING COMMENT 'Vendor description',
        is_active BOOLEAN COMMENT 'Whether vendor is active',
        created_ts TIMESTAMP COMMENT 'Creation timestamp'
    ) USING DELTA
    COMMENT 'Vendor registry - uses token overlap for entity resolution'
""")
print(f"  ✓ Ensured {vendor_table} exists")

# Create md_data_stream table if it doesn't exist
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {data_stream_table} (
        data_stream_id STRING NOT NULL COMMENT 'Unique data stream identifier',
        data_stream_name STRING NOT NULL COMMENT 'Canonical data stream name (shortest/normalized)',
        aliases ARRAY<STRING> COMMENT 'All variant names mapped to this data stream',
        data_stream_description STRING COMMENT 'Data stream description',
        is_active BOOLEAN COMMENT 'Whether data stream type is active',
        created_ts TIMESTAMP COMMENT 'Creation timestamp'
    ) USING DELTA
    COMMENT 'Data stream type registry - uses token overlap for entity resolution'
""")
print(f"  ✓ Ensured {data_stream_table} exists")

# Create md_study table if it doesn't exist (trial_id = study_id)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {study_table} (
        study_id STRING NOT NULL COMMENT 'Unique study identifier (same as trial_id)',
        study_title STRING COMMENT 'Study title/name',
        study_description STRING COMMENT 'Study description',
        status STRING COMMENT 'Study status',
        protocol_id STRING COMMENT 'Associated protocol document ID',
        created_by_principal STRING COMMENT 'User who created the study',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated the study',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp',
        databricks_job_id STRING COMMENT 'Job ID for lineage',
        databricks_job_name STRING COMMENT 'Job name for lineage',
        databricks_run_id STRING COMMENT 'Run ID for lineage'
    ) USING DELTA
    COMMENT 'Study registry - auto-populated from tsDTA trial_id'
""")
print(f"  ✓ Ensured {study_table} exists")


In [ ]:
# Cell 4: Collect All Names by parent_document_id
# This groups vendor/data_stream names from both tsDTA and OA by their common parent

print("\n" + "=" * 60)
print("Collecting names from source tables...")
print("=" * 60)

# Collect from Excel Raw table (includes trial_id for study creation)
df_excel_names = None
if spark.catalog.tableExists(excel_raw_table):
    df_excel_names = spark.sql(f"""
        SELECT 
            parent_document_id,
            trial_id,
            data_provider_name as vendor_name,
            data_stream_type as data_stream_name,
            'tsDTA' as source
        FROM {excel_raw_table}
        WHERE parent_document_id IS NOT NULL
          AND data_provider_name IS NOT NULL
          AND data_provider_name != 'UNKNOWN'
    """)
    print(f"  ✓ Found {df_excel_names.count()} rows from Excel Raw")
else:
    print(f"  ⚠ Excel Raw table does not exist: {excel_raw_table}")

# Collect from OA Raw table (if exists)
df_oa_names = None
if spark.catalog.tableExists(oa_raw_table):
    # OA raw may have different column names - check schema
    oa_columns = [c.name for c in spark.table(oa_raw_table).schema.fields]
    
    # Check for vendor/data_stream columns (they may not exist in raw OA)
    has_vendor = 'data_provider_name' in oa_columns or 'vendor_name' in oa_columns
    has_stream = 'data_stream_type' in oa_columns or 'data_stream_name' in oa_columns
    has_trial = 'trial_id' in oa_columns
    
    if has_vendor or has_stream:
        vendor_col = 'data_provider_name' if 'data_provider_name' in oa_columns else 'vendor_name' if 'vendor_name' in oa_columns else None
        stream_col = 'data_stream_type' if 'data_stream_type' in oa_columns else 'data_stream_name' if 'data_stream_name' in oa_columns else None
        
        select_parts = ["parent_document_id"]
        # Add trial_id if available
        if has_trial:
            select_parts.append("trial_id")
        else:
            select_parts.append("NULL as trial_id")
        if vendor_col:
            select_parts.append(f"{vendor_col} as vendor_name")
        else:
            select_parts.append("NULL as vendor_name")
        if stream_col:
            select_parts.append(f"{stream_col} as data_stream_name")
        else:
            select_parts.append("NULL as data_stream_name")
        select_parts.append("'OA' as source")
        
        df_oa_names = spark.sql(f"""
            SELECT {', '.join(select_parts)}
            FROM {oa_raw_table}
            WHERE parent_document_id IS NOT NULL
        """)
        print(f"  ✓ Found {df_oa_names.count()} rows from OA Raw")
    else:
        print(f"  ⚠ OA Raw table exists but no vendor/stream columns found")
else:
    print(f"  ⚠ OA Raw table does not exist: {oa_raw_table}")

# Union the sources
if df_excel_names is not None and df_oa_names is not None:
    df_all_names = df_excel_names.unionByName(df_oa_names)
elif df_excel_names is not None:
    df_all_names = df_excel_names
elif df_oa_names is not None:
    df_all_names = df_oa_names
else:
    print("\n❌ No source tables found with data. Exiting.")
    dbutils.notebook.exit("No source data")

print(f"\nTotal combined rows: {df_all_names.count()}")


In [ ]:
# Cell 5: Group Names by parent_document_id
# This gives us all variant names for each ZIP file

print("\n" + "=" * 60)
print("Grouping names by parent_document_id...")
print("=" * 60)

# Group vendors by parent_document_id
df_vendor_groups = df_all_names.filter(F.col("vendor_name").isNotNull()).groupBy("parent_document_id").agg(
    collect_set("vendor_name").alias("vendor_names"),
    collect_set("source").alias("sources")
)

# Group data streams by parent_document_id
df_stream_groups = df_all_names.filter(F.col("data_stream_name").isNotNull()).groupBy("parent_document_id").agg(
    collect_set("data_stream_name").alias("stream_names"),
    collect_set("source").alias("sources")
)

print(f"  Vendor groups: {df_vendor_groups.count()}")
print(f"  Stream groups: {df_stream_groups.count()}")

# Show groups with multiple names (these need normalization)
print("\nVendor groups with multiple variants:")
df_vendor_groups.filter(F.size("vendor_names") > 1).show(truncate=False)

print("\nData stream groups with multiple variants:")
df_stream_groups.filter(F.size("stream_names") > 1).show(truncate=False)


In [ ]:
# Cell 6: Apply Token Overlap Clustering
# For each group with multiple names, find which ones should be merged

print("\n" + "=" * 60)
print("Applying token overlap clustering...")
print("=" * 60)

# Collect all unique vendor names
all_vendor_names = [row.vendor_name for row in df_all_names.select("vendor_name").distinct().filter(F.col("vendor_name").isNotNull()).collect()]
all_stream_names = [row.data_stream_name for row in df_all_names.select("data_stream_name").distinct().filter(F.col("data_stream_name").isNotNull()).collect()]

print(f"Unique vendor names: {len(all_vendor_names)}")
print(f"Unique stream names: {len(all_stream_names)}")

# Cluster vendors using token overlap
vendor_clusters = cluster_similar_names(all_vendor_names, threshold=0.5)
print(f"\nVendor clusters formed: {len(vendor_clusters)}")
for i, cluster in enumerate(vendor_clusters):
    if len(cluster) > 1:
        canonical = min(cluster, key=len)
        print(f"  Cluster {i+1}: {cluster} -> canonical: '{canonical}'")

# Cluster data streams using token overlap
stream_clusters = cluster_similar_names(all_stream_names, threshold=0.5)
print(f"\nData stream clusters formed: {len(stream_clusters)}")
for i, cluster in enumerate(stream_clusters):
    if len(cluster) > 1:
        canonical = min(cluster, key=len)
        print(f"  Cluster {i+1}: {cluster} -> canonical: '{canonical}'")


In [ ]:
# Cell 7: MERGE - Insert New Records or Update Aliases
# Uses MERGE to handle both INSERT and UPDATE in a single statement
# Processes ALL clusters (including single-name clusters for new records)

print("\n" + "=" * 60)
print("MERGE: Creating/updating reference tables...")
print("=" * 60)

vendors_inserted = 0
vendors_updated = 0

# Process ALL vendor clusters (including single-name for new records)
for cluster in vendor_clusters:
    # Select canonical name (shortest)
    canonical = min(cluster, key=len)
    aliases = list(cluster)
    
    # Escape for SQL
    canonical_escaped = canonical.replace("'", "''")
    aliases_sql = ", ".join([f"'{a.replace(chr(39), chr(39)+chr(39))}'" for a in aliases])
    
    # Generate vendor_id for potential INSERT
    vendor_id = f"V-{str(uuid.uuid4())[:8].upper()}"
    
    # MERGE: Insert if not exists, Update aliases if exists
    # Match on: canonical name matches OR name is in existing aliases
    # Note: Using array_contains (scalar function) instead of EXISTS subquery (not supported in Delta MERGE)
    result = spark.sql(f"""
        MERGE INTO {vendor_table} AS target
        USING (
            SELECT 
                '{canonical_escaped}' AS vendor_name,
                array({aliases_sql}) AS new_aliases
        ) AS source
        ON UPPER(target.vendor_name) = UPPER(source.vendor_name)
           OR array_contains(target.aliases, source.vendor_name)
        WHEN MATCHED THEN
            UPDATE SET 
                aliases = array_distinct(array_union(COALESCE(target.aliases, array()), source.new_aliases))
        WHEN NOT MATCHED THEN
            INSERT (vendor_id, vendor_name, aliases, vendor_description, is_active, created_ts)
            VALUES (
                '{vendor_id}',
                source.vendor_name,
                source.new_aliases,
                'Auto-populated with token overlap normalization',
                true,
                current_timestamp()
            )
    """)
    
    # Log what happened
    if len(cluster) > 1:
        print(f"  ✓ MERGE vendor '{canonical}' (cluster of {len(cluster)}): {aliases}")
    
    vendors_inserted += 1

streams_inserted = 0
streams_updated = 0

# Process ALL data stream clusters
for cluster in stream_clusters:
    # Select canonical name (shortest)
    canonical = min(cluster, key=len)
    aliases = list(cluster)
    
    # Escape for SQL
    canonical_escaped = canonical.replace("'", "''")
    aliases_sql = ", ".join([f"'{a.replace(chr(39), chr(39)+chr(39))}'" for a in aliases])
    
    # Generate stream_id for potential INSERT
    stream_id = f"DS-{str(uuid.uuid4())[:8].upper()}"
    
    # MERGE: Insert if not exists, Update aliases if exists
    # Note: Using array_contains (scalar function) instead of EXISTS subquery (not supported in Delta MERGE)
    result = spark.sql(f"""
        MERGE INTO {data_stream_table} AS target
        USING (
            SELECT 
                '{canonical_escaped}' AS data_stream_name,
                array({aliases_sql}) AS new_aliases
        ) AS source
        ON UPPER(target.data_stream_name) = UPPER(source.data_stream_name)
           OR array_contains(target.aliases, source.data_stream_name)
        WHEN MATCHED THEN
            UPDATE SET 
                aliases = array_distinct(array_union(COALESCE(target.aliases, array()), source.new_aliases))
        WHEN NOT MATCHED THEN
            INSERT (data_stream_id, data_stream_name, aliases, data_stream_description, is_active, created_ts)
            VALUES (
                '{stream_id}',
                source.data_stream_name,
                source.new_aliases,
                'Auto-populated with token overlap normalization',
                true,
                current_timestamp()
            )
    """)
    
    # Log what happened
    if len(cluster) > 1:
        print(f"  ✓ MERGE stream '{canonical}' (cluster of {len(cluster)}): {aliases}")
    
    streams_inserted += 1

print(f"\n✅ MERGE complete:")
print(f"   Vendor clusters processed: {len(vendor_clusters)}")
print(f"   Data stream clusters processed: {len(stream_clusters)}")


In [ ]:
# Cell 7b: Create Studies from trial_ids
# trial_id is synonymous with study_id

print("\n" + "=" * 60)
print("Creating/updating studies from trial_ids...")
print("=" * 60)

# Collect distinct trial_ids from the data
all_trial_ids = [
    row.trial_id for row in df_all_names.select("trial_id").distinct()
    .filter(F.col("trial_id").isNotNull())
    .filter(F.col("trial_id") != "UNKNOWN")
    .filter(F.trim(F.col("trial_id")) != "")
    .collect()
] if "trial_id" in df_all_names.columns else []

print(f"Found {len(all_trial_ids)} distinct trial_id(s)")

studies_created = 0
for trial_id in all_trial_ids:
    # Escape for SQL
    trial_id_escaped = trial_id.replace("'", "''")
    
    # MERGE: Insert if study doesn't exist
    spark.sql(f"""
        MERGE INTO {study_table} AS target
        USING (SELECT '{trial_id_escaped}' AS study_id) AS source
        ON target.study_id = source.study_id
        WHEN NOT MATCHED THEN
            INSERT (study_id, study_title, study_description, status, protocol_id, 
                    created_by_principal, created_ts, last_updated_by_principal, last_updated_ts,
                    databricks_job_id, databricks_job_name, databricks_run_id)
            VALUES (
                '{trial_id_escaped}',
                'Auto-created from tsDTA: {trial_id_escaped}',
                'Study auto-populated from document processing',
                'IN_PROGRESS',
                NULL,
                '{created_by_principal}',
                current_timestamp(),
                '{created_by_principal}',
                current_timestamp(),
                '{databricks_job_id}',
                'normalize_reference_data',
                '{databricks_run_id}'
            )
    """)
    studies_created += 1

print(f"  ✓ Processed {studies_created} study/trial(s)")


In [ ]:
# Cell 8: Set Task Values for Downstream

# Set task values so DTA creation knows normalization completed
dbutils.jobs.taskValues.set(key="normalization_status", value="SUCCESS")
dbutils.jobs.taskValues.set(key="vendors_processed", value=str(len(vendor_clusters)))
dbutils.jobs.taskValues.set(key="streams_processed", value=str(len(stream_clusters)))
dbutils.jobs.taskValues.set(key="studies_processed", value=str(len(all_trial_ids)))
dbutils.jobs.taskValues.set(key="multi_alias_vendors", value=str(len([c for c in vendor_clusters if len(c) > 1])))
dbutils.jobs.taskValues.set(key="multi_alias_streams", value=str(len([c for c in stream_clusters if len(c) > 1])))

print("\n" + "=" * 60)
print("REFERENCE DATA CREATION & NORMALIZATION COMPLETE")
print("=" * 60)
print(f"Vendors processed: {len(vendor_clusters)} (multi-alias: {len([c for c in vendor_clusters if len(c) > 1])})")
print(f"Data streams processed: {len(stream_clusters)} (multi-alias: {len([c for c in stream_clusters if len(c) > 1])})")
print(f"Studies created: {len(all_trial_ids)}")
print(f"Job ID: {databricks_job_id}")
print(f"Run ID: {databricks_run_id}")


In [ ]:
# Cell 9: Verification - Show All Reference Data

print("\n" + "=" * 60)
print("Current Vendors:")
print("=" * 60)
spark.sql(f"SELECT vendor_id, vendor_name, aliases, is_active, created_ts FROM {vendor_table} ORDER BY created_ts DESC LIMIT 20").show(truncate=False)

print("\n" + "=" * 60)
print("Current Data Streams:")
print("=" * 60)
spark.sql(f"SELECT data_stream_id, data_stream_name, aliases, is_active, created_ts FROM {data_stream_table} ORDER BY created_ts DESC LIMIT 20").show(truncate=False)

print("\n" + "=" * 60)
print("Current Studies:")
print("=" * 60)
spark.sql(f"SELECT study_id, study_title, status, created_ts FROM {study_table} ORDER BY created_ts DESC LIMIT 20").show(truncate=False)
